In [8]:
import requests
import math
import json
from json.decoder import *
import pandas as pd
import numpy as np
import datetime
import sys

In [9]:
def haversine(lat1, lon1, lat2, lon2):
    R = 6378.137
    dLat = lat2 * math.pi / 180 - lat1 * math.pi / 180
    dLon = lon2 * math.pi / 180 - lon1 * math.pi / 180
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(lat1 * math.pi / 180) \
        * math.cos(lat2 * math.pi / 180) * math.sin(dLon/2) * math.sin(dLon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = R * c
    return d * 1000

In [73]:
train = pd.read_csv(r"\train.csv", ",", decimal=".")
test = pd.read_csv(r"\test.csv", ",", decimal=".")
atm_data = pd.concat([test, train], axis=0)
n = atm_data.shape[0]

In [70]:
overpass_url = "http://overpass.openstreetmap.fr/api/interpreter"
df = pd.DataFrame(columns=["atm",  "dist", "place", "category","comment"])
df.dtype={'atm': np.int64,  'dist': np.float64, 'place': np.str, 'category': np.str, 'comment': np.str }
r = 500
i=0

In [74]:
n

8765

In [71]:
start = datetime.datetime.now()
readed = 0
for index, row in atm_data.iterrows():
    
    atm = np.int64(row.loc["id"])
    origin_lat = row.loc["lat"]
    origin_lon = row.loc["long"]

    overpass_query = """
    [timeout:9000][maxsize:1073741824][out:json];
    (
       node["amenity"](around:{0},{1},{2});
       way["amenity"](around:{0},{1},{2});
       relation["amenity"](around:{0},{1},{2});
       node["shop"](around:{0},{1},{2});
       way["shop"](around:{0},{1},{2});
       relation["shop"](around:{0},{1},{2});
       node["tourism"](around:{0},{1},{2});
       way["tourism"](around:{0},{1},{2});
       relation["tourism"](around:{0},{1},{2});
       node["public_transport"](around:{0},{1},{2});
       way["public_transport"](around:{0},{1},{2});
       relation["public_transport"](around:{0},{1},{2});
       <;
       >;
    );
    out center;
    """.format(r, origin_lat, origin_lon)
    response = requests.get(overpass_url, params={'data': overpass_query})
    print (response)
    data = response.json()
    for el in data["elements"]:
        if ("center" in el):
            c = el["center"]
            if ("tags" in el.keys()):
                t =  el["tags"]
                name, what, category = None, None, None
                if ("name" in t.keys()): name = t["name"]
                else:
                    if ("brand" in t.keys()): name = t["brand"]
                    else:
                        if ("operator" in t.keys()): name = t["operator"]

                if ("public_transport" in t.keys()): 
                    what = t["public_transport"]
                    category = "public_transport"
                else:
                    if ("tourism" in t.keys()): 
                        what = t["tourism"]  
                        category = "tourism"
                    else:
                        if ("amenity" in t.keys()): 
                            what = t["amenity"]  
                            category = "amenity"
                        else: 
                            if ("shop" in t.keys()): 
                                what = t["shop"]
                                category = "shop"
                                if (what=="yes"): what = "shop"
                            else: continue
                dist = round(haversine(origin_lat, origin_lon, c["lat"], c["lon"]), 3)
                df = df.append(pd.DataFrame({"atm" : atm, "dist" : dist, "place" : what, "comment" : name, "category" : category}, index=[i]).loc[:,df.columns.tolist()])
                i+=1
    n = n-1
    readed+=1
    print (atm, ", left="+str(n), start +(datetime.datetime.now() - start)/readed * (n-readed))

<Response [200]>
8096 , left=8722 2018-12-21 02:13:06.856072
<Response [200]>
2570 , left=8721 2018-12-21 01:18:27.764773
<Response [200]>
2428 , left=8720 2018-12-21 01:41:28.567397
<Response [200]>
2799 , left=8719 2018-12-21 02:18:26.334031
<Response [200]>
250 , left=8718 2018-12-21 02:46:54.216246
<Response [200]>
5759 , left=8717 2018-12-21 02:50:42.716564
<Response [200]>
8274 , left=8716 2018-12-21 03:04:47.655316
<Response [200]>
6797 , left=8715 2018-12-21 03:26:24.480788
<Response [200]>
2218 , left=8714 2018-12-21 03:19:32.639811
<Response [200]>
8465 , left=8713 2018-12-21 03:18:20.368426
<Response [200]>
7972 , left=8712 2018-12-21 03:13:26.627104
<Response [200]>
2616 , left=8711 2018-12-21 03:14:27.529377
<Response [200]>
1610 , left=8710 2018-12-21 03:28:07.789138
<Response [200]>
2990 , left=8709 2018-12-21 03:21:14.953636
<Response [200]>
8610 , left=8708 2018-12-21 03:23:56.896112
<Response [200]>
7541 , left=8707 2018-12-21 03:17:45.777994
<Response [200]>
8785 , l

<Response [200]>
3249 , left=8587 2018-12-21 05:46:35.268244
<Response [200]>
6772 , left=8586 2018-12-21 05:45:37.378006
<Response [200]>
1469 , left=8585 2018-12-21 05:43:02.929282
<Response [200]>
5018 , left=8584 2018-12-21 05:41:42.098311
<Response [200]>
4962 , left=8583 2018-12-21 05:41:33.617047
<Response [200]>
8352 , left=8582 2018-12-21 05:40:50.528835
<Response [200]>
8034 , left=8581 2018-12-21 05:41:04.345474
<Response [200]>
74 , left=8580 2018-12-21 05:39:48.610820
<Response [200]>
4478 , left=8579 2018-12-21 05:40:16.642836
<Response [200]>
1896 , left=8578 2018-12-21 05:38:55.729639
<Response [200]>
8118 , left=8577 2018-12-21 05:37:22.652301
<Response [200]>
3284 , left=8576 2018-12-21 05:34:59.101270
<Response [200]>
8494 , left=8575 2018-12-21 05:34:13.335007
<Response [200]>
2099 , left=8574 2018-12-21 05:32:54.140386
<Response [200]>
4544 , left=8573 2018-12-21 05:32:53.019632
<Response [200]>
4839 , left=8572 2018-12-21 05:32:22.642036
<Response [200]>
1675 , le

<Response [200]>
506 , left=8452 2018-12-21 04:30:22.459603
<Response [200]>
6078 , left=8451 2018-12-21 04:29:43.964230
<Response [200]>
1145 , left=8450 2018-12-21 04:29:18.347562
<Response [200]>
3383 , left=8449 2018-12-21 04:28:58.994761
<Response [200]>
5813 , left=8448 2018-12-21 04:28:31.247048
<Response [200]>
5390 , left=8447 2018-12-21 04:27:52.479529
<Response [200]>
6863 , left=8446 2018-12-21 04:27:28.538851
<Response [200]>
6373 , left=8445 2018-12-21 04:27:01.520239
<Response [200]>
6498 , left=8444 2018-12-21 04:26:32.626216
<Response [200]>
831 , left=8443 2018-12-21 04:26:02.494882
<Response [200]>
7068 , left=8442 2018-12-21 04:25:30.490597
<Response [200]>
7567 , left=8441 2018-12-21 04:24:55.627508
<Response [200]>
7868 , left=8440 2018-12-21 04:24:21.080800
<Response [200]>
3376 , left=8439 2018-12-21 04:24:04.081766
<Response [200]>
3022 , left=8438 2018-12-21 04:23:52.608137
<Response [200]>
8098 , left=8437 2018-12-21 04:23:19.765702
<Response [200]>
4353 , le

<Response [200]>
1286 , left=8317 2018-12-21 03:51:44.501773
<Response [200]>
8515 , left=8316 2018-12-21 03:51:24.004415
<Response [200]>
4170 , left=8315 2018-12-21 03:51:28.548563
<Response [200]>
3028 , left=8314 2018-12-21 03:51:19.309576
<Response [200]>
1205 , left=8313 2018-12-21 03:51:11.708762
<Response [200]>
7639 , left=8312 2018-12-21 03:50:55.599995
<Response [200]>
6928 , left=8311 2018-12-21 03:50:48.043678
<Response [200]>
6926 , left=8310 2018-12-21 03:50:39.904387
<Response [200]>
650 , left=8309 2018-12-21 03:50:50.485841
<Response [200]>
1048 , left=8308 2018-12-21 03:50:45.865486
<Response [200]>
4203 , left=8307 2018-12-21 03:50:29.242836
<Response [200]>
8648 , left=8306 2018-12-21 03:50:39.078003
<Response [200]>
2650 , left=8305 2018-12-21 03:50:37.004275
<Response [200]>
1350 , left=8304 2018-12-21 03:50:24.915221
<Response [200]>
1452 , left=8303 2018-12-21 03:50:18.497748
<Response [200]>
8220 , left=8302 2018-12-21 03:50:01.087108
<Response [200]>
7959 , l

<Response [200]>
2753 , left=8182 2018-12-21 03:51:25.854223
<Response [200]>
4298 , left=8181 2018-12-21 03:51:11.381565
<Response [200]>
7586 , left=8180 2018-12-21 03:50:55.302484
<Response [200]>
3691 , left=8179 2018-12-21 03:50:55.117666
<Response [200]>
8521 , left=8178 2018-12-21 03:50:31.771830
<Response [200]>
394 , left=8177 2018-12-21 03:50:18.958783
<Response [200]>
1110 , left=8176 2018-12-21 03:50:09.468475
<Response [200]>
941 , left=8175 2018-12-21 03:50:05.868595
<Response [200]>
5855 , left=8174 2018-12-21 03:49:50.556011
<Response [200]>
8529 , left=8173 2018-12-21 03:49:19.492126
<Response [200]>
1134 , left=8172 2018-12-21 03:49:11.670757
<Response [200]>
4729 , left=8171 2018-12-21 03:48:54.586200
<Response [200]>
1478 , left=8170 2018-12-21 03:48:36.388324
<Response [200]>
559 , left=8169 2018-12-21 03:48:38.217251
<Response [200]>
3823 , left=8168 2018-12-21 03:48:31.036451
<Response [200]>
6302 , left=8167 2018-12-21 03:48:19.396789
<Response [200]>
5774 , lef

...

<Response [200]>
5248 , left=611 2018-12-20 15:08:36.788750
<Response [200]>
5254 , left=610 2018-12-20 15:08:23.854918
<Response [200]>
5257 , left=609 2018-12-20 15:08:14.963201
<Response [200]>
5260 , left=608 2018-12-20 15:08:08.427758
<Response [200]>
5264 , left=607 2018-12-20 15:08:00.308248
<Response [200]>
5267 , left=606 2018-12-20 15:07:55.913682
<Response [200]>
5270 , left=605 2018-12-20 15:07:49.709955
<Response [200]>
5274 , left=604 2018-12-20 15:07:40.132501
<Response [200]>
5293 , left=603 2018-12-20 15:07:31.012296
<Response [200]>
5296 , left=602 2018-12-20 15:07:24.470064
<Response [200]>
5298 , left=601 2018-12-20 15:07:15.295810
<Response [200]>
5299 , left=600 2018-12-20 15:07:07.843251
<Response [200]>
5324 , left=599 2018-12-20 15:06:58.569486
<Response [200]>
5333 , left=598 2018-12-20 15:06:46.502080
<Response [200]>
5341 , left=597 2018-12-20 15:06:38.437494
<Response [200]>
5350 , left=596 2018-12-20 15:06:34.009901
<Response [200]>
5353 , left=595 2018-12

<Response [200]>
6280 , left=474 2018-12-20 14:50:49.140836
<Response [200]>
6281 , left=473 2018-12-20 14:50:41.254175
<Response [200]>
6282 , left=472 2018-12-20 14:50:33.468277
<Response [200]>
6283 , left=471 2018-12-20 14:50:26.841436
<Response [200]>
6284 , left=470 2018-12-20 14:50:20.526199
<Response [200]>
6285 , left=469 2018-12-20 14:50:14.055706
<Response [200]>
6288 , left=468 2018-12-20 14:50:07.624512
<Response [200]>
6296 , left=467 2018-12-20 14:49:58.350717
<Response [200]>
6297 , left=466 2018-12-20 14:49:49.434232
<Response [200]>
6300 , left=465 2018-12-20 14:49:39.269975
<Response [200]>
6301 , left=464 2018-12-20 14:49:30.218871
<Response [200]>
6307 , left=463 2018-12-20 14:49:16.847269
<Response [200]>
6308 , left=462 2018-12-20 14:49:07.808735
<Response [200]>
6309 , left=461 2018-12-20 14:48:58.652234
<Response [200]>
6316 , left=460 2018-12-20 14:48:53.154328
<Response [200]>
6326 , left=459 2018-12-20 14:48:36.105886
<Response [200]>
6367 , left=458 2018-12

<Response [200]>
7067 , left=337 2018-12-20 14:32:02.859025
<Response [200]>
7073 , left=336 2018-12-20 14:31:56.151810
<Response [200]>
7080 , left=335 2018-12-20 14:31:48.800607
<Response [200]>
7089 , left=334 2018-12-20 14:31:42.158176
<Response [200]>
7109 , left=333 2018-12-20 14:31:34.952039
<Response [200]>
7110 , left=332 2018-12-20 14:31:29.043405
<Response [200]>
7112 , left=331 2018-12-20 14:31:21.160474
<Response [200]>
7115 , left=330 2018-12-20 14:31:14.978504
<Response [200]>
7117 , left=329 2018-12-20 14:31:05.538786
<Response [200]>
7123 , left=328 2018-12-20 14:30:58.606801
<Response [200]>
7124 , left=327 2018-12-20 14:30:52.740064
<Response [200]>
7125 , left=326 2018-12-20 14:30:46.131463
<Response [200]>
7132 , left=325 2018-12-20 14:30:39.393994
<Response [200]>
7133 , left=324 2018-12-20 14:30:29.450986
<Response [200]>
7134 , left=323 2018-12-20 14:30:23.852052
<Response [200]>
7135 , left=322 2018-12-20 14:30:14.545657
<Response [200]>
7136 , left=321 2018-12

<Response [200]>
7981 , left=200 2018-12-20 14:14:23.645336
<Response [200]>
7987 , left=199 2018-12-20 14:14:15.389911
<Response [200]>
8000 , left=198 2018-12-20 14:14:08.258139
<Response [200]>
8003 , left=197 2018-12-20 14:13:54.513189
<Response [200]>
8008 , left=196 2018-12-20 14:13:46.546825
<Response [200]>
8022 , left=195 2018-12-20 14:13:39.513405
<Response [200]>
8026 , left=194 2018-12-20 14:13:35.155616
<Response [200]>
8041 , left=193 2018-12-20 14:13:28.131367
<Response [200]>
8042 , left=192 2018-12-20 14:13:21.474134
<Response [200]>
8043 , left=191 2018-12-20 14:13:14.450173
<Response [200]>
8046 , left=190 2018-12-20 14:13:06.658756
<Response [200]>
8048 , left=189 2018-12-20 14:12:58.157746
<Response [200]>
8049 , left=188 2018-12-20 14:12:45.941713
<Response [200]>
8059 , left=187 2018-12-20 14:12:38.198356
<Response [200]>
8062 , left=186 2018-12-20 14:12:30.362894
<Response [200]>
8070 , left=185 2018-12-20 14:12:21.424548
<Response [200]>
8078 , left=184 2018-12

3695 , left=63 2018-12-20 13:55:11.467832
<Response [200]>
3696 , left=62 2018-12-20 13:55:03.295923
<Response [200]>
3749 , left=61 2018-12-20 13:54:57.136228
<Response [200]>
3751 , left=60 2018-12-20 13:54:52.895518
<Response [200]>
3763 , left=59 2018-12-20 13:54:46.943821
<Response [200]>
3765 , left=58 2018-12-20 13:54:39.331585
<Response [200]>
3773 , left=57 2018-12-20 13:54:22.214853
<Response [200]>
3776 , left=56 2018-12-20 13:54:14.996195
<Response [200]>
3777 , left=55 2018-12-20 13:54:07.682818
<Response [200]>
3778 , left=54 2018-12-20 13:54:01.644441
<Response [200]>
3789 , left=53 2018-12-20 13:53:48.471760
<Response [200]>
3794 , left=52 2018-12-20 13:53:44.010148
<Response [200]>
3799 , left=51 2018-12-20 13:53:33.084090
<Response [200]>
3800 , left=50 2018-12-20 13:53:25.571044
<Response [200]>
3801 , left=49 2018-12-20 13:53:18.471886
<Response [200]>
3808 , left=48 2018-12-20 13:53:06.144846
<Response [200]>
3820 , left=47 2018-12-20 13:52:59.519231
<Response [200

In [72]:
df.to_csv(r"\near_atm.csv", ",", decimal=".")